## Lab5

Filip Katulski

In [17]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
import pandas as pd 
import warnings

## Gaussowski Naiwny Klasyfikator Bayesa

### Zadania 1, 2:

Klasa NaiveBayesClassifier zawiera metody słuzące do wyliczenia Naiwnego Bayesa dla zadanego zbioru danych.

Metody mean, probability, stddev, fit są wyliczane dla kazdej z zadanych klas (target). 

Dla obliczeć prawdpodobieństwa a posteriori uzyłem formuły wyprowadzonej jako suma logarytmów, wyprowadzenie [tutaj](https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Multinomial_naive_Bayes) 

In [15]:
class NaiveBayesClassifier:
    # Train, Test jako Pandas Dataframe, mozliwosc uzywania funkcji groupby, apply, to_numpy.

    def calculate_means(self, train, target):
        self.means = train.groupby(target).apply(np.mean).to_numpy()

    def calculate_variances(self, train, target):
        self.vars = train.groupby(target).apply(np.var).to_numpy()

    def calculate_priors(self, train, target):
        # shape[0] zwraca ilosc elementow dla danego targetu 
        self.priors = (train.groupby(target).apply(lambda x: len(x))/train.shape[0]).to_numpy()
  
    # Zgodnie ze wzorem Eq. 5 
    def gaussian(self, x, mean, variance):
        return np.exp((-1/2)*((x - mean[:-1])**2) / (2 * variance[:-1])) / np.sqrt(2 * np.pi * variance[:-1])

    def posterior_probability(self, x):
        posteriors = []
        for i in range(len(self.classes)):
            prior = np.log(self.priors[i])
            # iloczyn jako suma logarytmow, link do wyprowadzenia powyzej 
            sub_sum_logs = np.sum(np.log(self.gaussian(x, self.means[i], self.vars[i])))
            posterior = prior + sub_sum_logs

            posteriors.append(posterior)
        
            # maksymalna wartość praw. a posteriori
        return self.classes[np.argmax(posteriors)]

    def fit(self, train, target):
        self.calculate_means(train, target)
        self.calculate_variances(train, target)
        self.calculate_priors(train, target)
        self.classes = np.unique(train[target])

    # założenie dla podzbiorów "test" - ostatnia kolumna to "target"
    def predict(self, test):
        return [self.posterior_probability(x) for x in test.to_numpy()]


### Zadanie 3

Test implementacji dla zbioru Iris

In [13]:
def accuracy(predicted, actual):
  return np.sum(predicted == actual) / len(predicted)

def error(predicted, actual):
  return 1.0 - accuracy(predicted, actual)

In [25]:
# Numpy zwraca ostrzezenia odnosnie nowej wersji niektorych funkcji, 
# zostały one wyciszone. 
warnings.filterwarnings('ignore')

iris_df= load_iris(as_frame=True)
errors = []
for iteration in range(20):
    train, test = train_test_split(iris_df['frame'], test_size=0.4)
    X_test, y_test = test.drop(columns=['target']), test['target']
    cls = NaiveBayesClassifier()
    cls.fit(train, 'target')
    predictions = cls.predict(X_test)
    errors.append(error(predictions, y_test))
    print("|Iteration: {0} \n error: {1}".format(iteration, errors[iteration]))

print("Mean error:", np.mean(errors))
print("Error variance:", np.var(errors))

|Iteration: 0 
 error: 0.01666666666666672
|Iteration: 1 
 error: 0.01666666666666672
|Iteration: 2 
 error: 0.050000000000000044
|Iteration: 3 
 error: 0.050000000000000044
|Iteration: 4 
 error: 0.050000000000000044
|Iteration: 5 
 error: 0.050000000000000044
|Iteration: 6 
 error: 0.033333333333333326
|Iteration: 7 
 error: 0.01666666666666672
|Iteration: 8 
 error: 0.050000000000000044
|Iteration: 9 
 error: 0.08333333333333337
|Iteration: 10 
 error: 0.033333333333333326
|Iteration: 11 
 error: 0.06666666666666665
|Iteration: 12 
 error: 0.050000000000000044
|Iteration: 13 
 error: 0.050000000000000044
|Iteration: 14 
 error: 0.050000000000000044
|Iteration: 15 
 error: 0.08333333333333337
|Iteration: 16 
 error: 0.050000000000000044
|Iteration: 17 
 error: 0.06666666666666665
|Iteration: 18 
 error: 0.050000000000000044
|Iteration: 19 
 error: 0.033333333333333326
Mean error: 0.04750000000000003
Error variance: 0.000340972222222222


### Zadanie 4

Zbiór Wine. StandardScaler do standaryzacji danych wejściowych, PCA do redukcji wymiarowości.

In [26]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA